In [ ]:
## Example of pulling API data via Django. 
# in a file named "services.py"

def get_books(year, author):
    url = 'http://api.example.com/books' 
    params = {'year': year, 'author': author}
    r = requests.get(url, params=params)
    books = r.json()
    books_list = {'books':books['results']}
    return books_list

In [ ]:
# in a file named "views.py"

import services
class BooksPage(generic.TemplateView):
    def get(self,request):
        books_list = services.get_books('2009', 'edwards')
        return render(request,'books.html',books_list)

In [ ]:
## Another Example using simple restAPI freegeoip.net
# in a file named "urls.py"

from django.urls import path
from . import views

urlpatterns = [
    path('', views.home, name='home'),
]

In [ ]:
# in a file named "views.py"

from django.shortcuts import render
import requests

def home(request):
    response = requests.get('http://freegeoip.net/json/')
    geodata = response.json()
    return render(request, 'core/home.html', {
        'ip': geodata['ip'],
        'country': geodata['country_name']
    })

In [ ]:
# in a file named core/home.html

{% extends 'base.html' %}

{% block content %}
  <h2>GEO API</h2>
  <p>Your ip address is <strong>{{ ip }}</strong>, and you are probably in <strong>{{ country }}</strong> right now.</p>
{% endblock %}

In [ ]:
## Third exmaple for inputting an API, Using Oxford Dictionary API.

# First install decouple
pip install python-decouple
# Then create your .env folder in the project root, include API keys and application ID.
# In the .env folder
OXFORD_APP_ID=EIiHAFqo
OXFORD_APP_KEY=nJzxp5Fk21v7D9ASIWd8Vkhs47C0j6cS

In [ ]:
# In a file named settings.py
from decouple import config

# other settings... SECRET_KEY, DEBUG, INSTALLED_APPS etc etc

# at the bottom of the settings.py file:

OXFORD_APP_ID = config('OXFORD_APP_ID', default='')
OXFORD_APP_KEY = config('OXFORD_APP_KEY', default='')

In [ ]:
# In a file named forms.py
from django import forms
from django.conf import settings
import requests

class DictionaryForm(forms.Form):
    word = forms.CharField(max_length=100)

    def search(self):
        result = {}
        word = self.cleaned_data['word']
        endpoint = 'https://od-api.oxforddictionaries.com/api/v1/entries/{source_lang}/{word_id}'
        url = endpoint.format(source_lang='en', word_id=word)
        headers = {'app_id': settings.OXFORD_APP_ID, 'app_key': settings.OXFORD_APP_KEY}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:  # SUCCESS
            result = response.json()
            result['success'] = True
        else:
            result['success'] = False
            if response.status_code == 404:  # NOT FOUND
                result['message'] = 'No entry found for "%s"' % word
            else:
                result['message'] = 'The Oxford API is not available at the moment. Please try again later.'
        return result
    
## Only change potentially word, endpoint, url, and headers. Maybe repsonse. Change the response message.

In [ ]:
# In a file named views.py
from django.shortcuts import render
from .forms import DictionaryForm

def oxford(request):
    search_result = {}
    if 'word' in request.GET:
        form = DictionaryForm(request.GET)
        if form.is_valid():
            search_result = form.search()
    else:
        form = DictionaryForm()
    return render(request, 'core/oxford.html', {'form': form, 'search_result': search_result})

In [ ]:
# In a file named core/oxford.html  
# Using oxford dictionary API
from django.shortcuts import render
from .forms import DictionaryForm

def oxford(request):
    search_result = {}
    if 'word' in request.GET:
        form = DictionaryForm(request.GET)
        if form.is_valid():
            search_result = form.search()
    else:
        form = DictionaryForm()
    return render(request, 'core/oxford.html', {'form': form, 'search_result': search_result})